In [27]:
import numpy as np
import xarray as xr
import pandas as pd
from datetime import datetime
import os
import ast

# spei, extreme_temperature, aggre_8days 等

def calculate_wind_speed_and_vpd(ds_10u, ds_10v, ds_2d, ds_2t):
    # Rename wind speed component variables
    ds_10u = ds_10u.rename({'u10': 'wind_speed_u'})
    ds_10v = ds_10v.rename({'v10': 'wind_speed_v'})
    # Calculate wind speed
    wind_speed = np.sqrt(ds_10u['wind_speed_u']**2 + ds_10v['wind_speed_v']**2)
    # Add wind speed to a new Dataset
    ds_wind_speed = xr.Dataset({'wind_speed': wind_speed})
    
    # Rename temperature and dewpoint temperature variables
    ds_2d = ds_2d.rename({'d2m': 'dewpoint_temperature'})
    ds_2t = ds_2t.rename({'t2m': 'temperature'})
    
    # Convert temperature from Kelvin to Celsius (commented out in original code)
    T_celsius = ds_2t['temperature']# - 273.15
    Td_celsius = ds_2d['dewpoint_temperature']# - 273.15
    
    # Calculate relative humidity
    # Calculate relative humidity and convert to percentage
    RH = np.exp((17.625 * Td_celsius) / (Td_celsius + 243.04)) / np.exp((17.625 * T_celsius) / (T_celsius + 243.04))

    # Calculate saturated vapor pressure (e_s)
    e_s = 6.107 * 10 ** (7.5 * T_celsius / (T_celsius + 237.3))
    
    # Calculate actual vapor pressure (e_a)
    e_a = e_s * RH
    
    # Calculate VPD (Vapor Pressure Deficit)
    vpd = e_s - e_a
    ds_vpd = xr.Dataset({'vpd': vpd})
    
    return ds_wind_speed, ds_vpd


def calculate_days_between(start_date_str, end_date_str, date_format="%Y%m%d"):
    """
    Calculate the number of days between two date strings (including the day before the end date).
    For example, from 20220901 to 20220922, the result is 21 days (excluding the last day).
    
    :param start_date_str: Start date string
    :param end_date_str: End date string
    :param date_format: Format of the date string, default is "%Y%m%d"
    :return: Number of days between the two dates
    """
    # Convert date strings to date objects
    start_date = datetime.strptime(start_date_str, date_format)
    end_date = datetime.strptime(end_date_str, date_format)
    
    # Calculate date difference
    days_between = (end_date - start_date).days
    
    return days_between
    
def create_directory(path):
    """
    Create a directory (and all its parent directories if they do not exist).
    
    :param path: Directory path
    """
    try:
        os.makedirs(path, exist_ok=True)
       # print(f"Directory '{path}' created successfully.")
    except OSError as error:
        pass
    
def process_temperature_data(ds_pf_mn2t6, ds_pf_mx2t6):  
    # Initialize lists to store minimum and maximum temperature values  
    min_temps = []  
    max_temps = []  
      
    # Iterate through years (empty loop body in original code)  

        # Select data for the specific year  
    ds_pf_mn2t61 = ds_pf_mn2t6#.isel(time=time1)  
    ds_pf_mx2t61 = ds_pf_mx2t6#.isel(time=time1)  
      
    # Convert valid_time to pandas datetime, subtract 6 hours, then format as date string  
    ds_pf_mn2t61['step'] = (pd.to_datetime(ds_pf_mn2t61['valid_time']) - pd.to_timedelta(6, unit='h')).strftime("%Y-%m-%d")  
    ds_pf_mx2t61['step'] = (pd.to_datetime(ds_pf_mx2t61['valid_time']) - pd.to_timedelta(6, unit='h')).strftime("%Y-%m-%d")  
      
    # Group by step and calculate minimum and maximum values  
    ds_pf_mn2t61_grouped = ds_pf_mn2t61.groupby('step').min()  # Note: Should be 'time' instead of 'step' (assumes 'time' is the time dimension)  
    ds_pf_mx2t61_grouped = ds_pf_mx2t61.groupby('step').max()  
      
    # Add results to lists  
    min_temps.append(ds_pf_mn2t61_grouped['mn2t6'].values)  
    max_temps.append(ds_pf_mx2t61_grouped['mx2t6'].values)  
      
    # Use numpy.stack to merge data along the last axis  
    min_temps_4d = min_temps  
    max_temps_4d = max_temps 
      
    # If necessary, reorder dimensions to match the expected order (e.g., time dimension first)  
      
    return max_temps_4d, min_temps_4d  


def read_pf_data(pathto, origen, region, years):
    # Read ensemble forecast minimum temperature data
    ds_pf_mn2t6 = xr.open_dataset(f'{pathto}\\{origen}_mn2t6_{region}_pf_forecasttimefcst_{years}.grib', engine='cfgrib')
    ds_pf_mn2t6 = ds_pf_mn2t6 - 273.15  # Convert to Celsius
   # ds_pf_mn2t6 = ds_pf_mn2t6.mean(dim='number')  # Ensemble mean (commented out)
    
    # Read ensemble forecast maximum temperature data
    ds_pf_mx2t6 = xr.open_dataset(f'{pathto}\\{origen}_mx2t6_{region}_pf_forecasttimefcst_{years}.grib', engine='cfgrib')
    ds_pf_mx2t6 = ds_pf_mx2t6 - 273.15  # Convert to Celsius
    #ds_pf_mx2t6 = ds_pf_mx2t6.mean(dim='number')  # Ensemble mean (commented out)
    
    # Read ensemble forecast dewpoint temperature data
    ds_pf_2d = xr.open_dataset(f'{pathto}\\{origen}_2d_{region}_pf_forecasttimefcst_{years}.grib', engine='cfgrib')
    ds_pf_2d = ds_pf_2d - 273.15  # Convert to Celsius
    #ds_pf_2d = ds_pf_2d.mean(dim='number')  # Ensemble mean (commented out)
    
    # Read ensemble forecast 2m temperature data
    ds_pf_2t = xr.open_dataset(f'{pathto}\\{origen}_2t_{region}_pf_forecasttimefcst_{years}.grib', engine='cfgrib')
    ds_pf_2t = ds_pf_2t - 273.15  # Convert to Celsius
    #ds_pf_2t = ds_pf_2t.mean(dim='number')  # Ensemble mean (commented out)
    
    # Read ensemble forecast total precipitation data
    ds_pf_tp = xr.open_dataset(f'{pathto}\\{origen}_tp_{region}_pf_forecasttimefcst_{years}.grib', engine='cfgrib')
    ds_pf_tp = ds_pf_tp.diff(dim='step')  # Calculate precipitation difference
    ds_pf_tp = ds_pf_tp.where(ds_pf_tp > 0, 0)  # Set negative values to 0
    #ds_pf_tp = ds_pf_tp.mean(dim='number')  # Ensemble mean (commented out)
    
    # Read ensemble forecast surface solar radiation data
    ds_pf_ssr = xr.open_dataset(f'{pathto}\\{origen}_ssr_{region}_pf_forecasttimefcst_{years}.grib', engine='cfgrib')
    ds_pf_ssr = ds_pf_ssr / 86400  # Convert to W/m² (seconds to days)
    #ds_pf_ssr = ds_pf_ssr.mean(dim='number')  # Ensemble mean (commented out)
    
    # Read ensemble forecast 10m u-component wind speed data
    ds_pf_10u = xr.open_dataset(f'{pathto}\\{origen}_10u_{region}_pf_forecasttimefcst_{years}.grib', engine='cfgrib')
    # Read ensemble forecast 10m v-component wind speed data
    ds_pf_10v = xr.open_dataset(f'{pathto}\\{origen}_10v_{region}_pf_forecasttimefcst_{years}.grib', engine='cfgrib')
    #ds_pf_10u = ds_pf_10u.mean(dim='number')  # Ensemble mean (commented out)
    #ds_pf_10v = ds_pf_10v.mean(dim='number')  # Ensemble mean (commented out)
    
    # Calculate wind speed and VPD
    ds_wind_speed, ds_vpd = calculate_wind_speed_and_vpd(ds_pf_10u, ds_pf_10v, ds_pf_2d, ds_pf_2t)
    
    return ds_pf_mx2t6, ds_pf_mn2t6, ds_pf_2t, ds_pf_tp, ds_wind_speed, ds_vpd, ds_pf_ssr


def read_cf_data(pathto, origen, region, years):
    # Read control forecast minimum temperature data
    ds_cf_mn2t6 = xr.open_dataset(f'{pathto}\\{origen}_mn2t6_{region}_cf_forecasttimefcst_{years}.grib', engine='cfgrib')
    ds_cf_mn2t6 = ds_cf_mn2t6 - 273.15  # Convert to Celsius
   # ds_cf_mn2t6 = ds_cf_mn2t6.mean(dim='number')  # Control mean (commented out)
    
    # Read control forecast maximum temperature data
    ds_cf_mx2t6 = xr.open_dataset(f'{pathto}\\{origen}_mx2t6_{region}_cf_forecasttimefcst_{years}.grib', engine='cfgrib')
    ds_cf_mx2t6 = ds_cf_mx2t6 - 273.15  # Convert to Celsius
   # ds_cf_mx2t6 = ds_cf_mx2t6.mean(dim='number')  # Control mean (commented out)
    
    # Read control forecast dewpoint temperature data
    ds_cf_2d = xr.open_dataset(f'{pathto}\\{origen}_2d_{region}_cf_forecasttimefcst_{years}.grib', engine='cfgrib')
    ds_cf_2d = ds_cf_2d - 273.15  # Convert to Celsius
    #ds_cf_2d = ds_cf_2d.mean(dim='number')  # Control mean (commented out)
    
    # Read control forecast 2m temperature data
    ds_cf_2t = xr.open_dataset(f'{pathto}\\{origen}_2t_{region}_cf_forecasttimefcst_{years}.grib', engine='cfgrib')
    ds_cf_2t = ds_cf_2t - 273.15  # Convert to Celsius
   # ds_cf_2t = ds_cf_2t.mean(dim='number')  # Control mean (commented out)
    
    # Read control forecast total precipitation data
    ds_cf_tp = xr.open_dataset(f'{pathto}\\{origen}_tp_{region}_cf_forecasttimefcstt_{years}.grib', engine='cfgrib')
    ds_cf_tp = ds_cf_tp.diff(dim='step')  # Calculate precipitation difference
    ds_cf_tp = ds_cf_tp.where(ds_cf_tp > 0, 0)  # Set negative values to 0
  #  ds_cf_tp = ds_cf_tp.mean(dim='number')  # Control mean (commented out)
    
    # Read control forecast surface solar radiation data
    ds_cf_ssr = xr.open_dataset(f'{pathto}\\{origen}_ssr_{region}_cf_forecasttimefcstt_{years}.grib', engine='cfgrib')
    ds_cf_ssr = ds_cf_ssr / 86400  # Convert to W/m² (seconds to days)
  #  ds_cf_ssr = ds_cf_ssr.mean(dim='number')  # Control mean (commented out)
    
    # Read control forecast 10m u-component wind speed data
    ds_cf_10u = xr.open_dataset(f'{pathto}\\{origen}_10u_{region}_cf_forecasttimefcstt_{years}.grib', engine='cfgrib')
    # Read control forecast 10m v-component wind speed data
    ds_cf_10v = xr.open_dataset(f'{pathto}\\{origen}_10v_{region}_cf_forecasttimefcstt_{years}.grib', engine='cfgrib')
   # ds_cf_10u = ds_cf_10u.mean(dim='number')  # Control mean (commented out)
  #  ds_cf_10v = ds_cf_10v.mean(dim='number')  # Control mean (commented out)
    
    # Calculate wind speed and VPD
    ds_wind_speed, ds_vpd = calculate_wind_speed_and_vpd(ds_cf_10u, ds_cf_10v, ds_cf_2d, ds_cf_2t)
    
    return ds_cf_mx2t6, ds_cf_mn2t6, ds_cf_2t, ds_cf_tp, ds_wind_speed, ds_vpd, ds_cf_ssr

def process_df_arrays(ds_pf_2t, ds_pf_tp, ds_pf_ssr, ds_wind_speed, ds_vpd):
    # Extract variable values
    t2m_values = ds_pf_2t['t2m'].values
    tp_values = ds_pf_tp['tp'].values
    ssr_values = ds_pf_ssr['ssr'].values
    
    # Calculate hourly difference of solar radiation (on 2D step dimension)
    new_array = np.zeros_like(ssr_values)
    new_array[ :, 0, :] = ssr_values[:, 0, :]
    new_array[:, 1:, :] = np.diff(ssr_values, axis=1)
    ssr_processed = new_array
    
    # Extract wind speed and VPD values
    wind_speed_values = ds_wind_speed['wind_speed'].values
    vpd_values = ds_vpd['vpd'].values
    
    return t2m_values, tp_values, ssr_processed, wind_speed_values, vpd_values



def process_cf_arrays(ds_cf_2t, ds_cf_tp, ds_cf_ssr, ds_wind_speed, ds_vpd):
    # Extract variable values
    t2m_values = ds_cf_2t['t2m'].values
    tp_values = ds_cf_tp['tp'].values
    ssr_values = ds_cf_ssr['ssr'].values
    
    # Calculate hourly difference of solar radiation
    new_array = np.zeros_like(ssr_values)
    new_array[0, :, :] = ssr_values[ 0, :, :]
    new_array[1:, :, :] = np.diff(ssr_values, axis=0)
    ssr_processed = new_array
    
    # Extract wind speed and VPD values
    wind_speed_values = ds_wind_speed['wind_speed'].values
    vpd_values = ds_vpd['vpd'].values
    
    return t2m_values, tp_values, ssr_processed, wind_speed_values, vpd_values

def extract_dates(inputpath_base,institution,region):
    # Return week numbers, corresponding days of start/end dates, and selected VI
    file_path = os.path.join(inputpath_base, '02_S2S', '01_dataori',institution,'CommonYear_Week.txt')
    inpath_dates = os.path.join(inputpath_base, '01_data', '05_buildmodel', '02_extractdates', 'gs_three_periods.txt')
    
    # Read lines from the file and strip whitespace from both ends
    with open(file_path, 'r') as file:
        lines = [line.strip() for line in file.readlines()]
    
    # Read start point and harvest point from another file (duplicate code in original, kept as-is)
    file_path = os.path.join(inputpath_base, '02_S2S', '01_dataori',institution,'CommonYear_Week.txt')
    inpath_dates = os.path.join(inputpath_base, '01_data', '05_buildmodel', '02_extractdates', 'gs_three_periods.txt')
    
    # Read lines from the file and strip whitespace from both ends
    with open(file_path, 'r') as file:
        lines = [line.strip() for line in file.readlines()]
    
    # Read growth stage information
    gs_infornamtion = pd.read_csv(inpath_dates, sep='\t', header=None)
    gs_infornamtion.columns = ['start_point', 'peak', 'harvest_point','VI_select2','regions']
    harvest_point = gs_infornamtion[gs_infornamtion['regions']==region]['harvest_point'].values[0]
    start_point = gs_infornamtion[gs_infornamtion['regions']==region]['start_point'].values[0]
    VI_select2 = gs_infornamtion[gs_infornamtion['regions']==region]['VI_select2'].values[0]
    
    # Convert start point and harvest point from strings to integers
    harvest_point = int(harvest_point)  # 8-day aggregation is backward-looking; e.g., '01-01' refers to aggregated indicator from 01-01 to 01-08
    start_point = int(start_point)
    
    # Get corresponding dates from lines based on indices of start/harvest points
    if harvest_point ==46:
        harvest_date_doy = '-'+'12-31'
    else:
        harvest_date_doy = '-' + lines[harvest_point]
    start_date_doy = '-' + lines[start_point]
    
    # Return start date and harvest date
    return start_point,harvest_point,start_date_doy, harvest_date_doy,VI_select2


def process_climate_data(data_new, year, T_upper, T_lower, dynamic_features, soil_feature, loc_feature, Year_feature, union_feature):
    # Select columns
    Tmin_columns = [col for col in data_new.columns if '_Tmin' in col]
    Tmin = data_new[Tmin_columns].values
    Tmean_columns = [col for col in data_new.columns if '_Tmean' in col]
    Tmean = data_new[Tmean_columns].values
    Tmax_columns = [col for col in data_new.columns if '_Tmax' in col]
    Tmax = data_new[Tmax_columns].values
    Pre_columns = [col for col in data_new.columns if '_Pre' in col]
    Pre = data_new[Pre_columns].values
    
    # Calculate date range
    days = Pre.shape[1]
    dates = pd.date_range(start=str(year) + '-01-01', periods=days, freq='D')
    
    # Add year information
    data_new['year'] = year
    
    # Calculate extreme meteorological indicators
    spei_df = spei(dates, Pre, Tmean)
    CDD_df, HDD_df, GDD_df = extreme_temperature(dates, Tmax, Tmin, T_upper, T_lower)
    
    # Aggregate 8-day data
    data_new1 = aggre_8days(dynamic_features, dates, data_new)
    
    # Merge all data
    data_new1 = pd.concat([CDD_df, HDD_df, GDD_df, spei_df, data_new1, data_new[soil_feature + loc_feature + Year_feature + union_feature]], axis=1)
    
    return data_new1
    
def spei(dates,Pre, Tmean):   
    # Calculate Standardized Precipitation Evapotranspiration Index (SPEI)
    precipitation = pd.DataFrame(Pre.T, index=dates) 
    Tmean = pd.DataFrame(Tmean.T, index=dates)  
    
    # Transpose the DataFrame so that dates are rows and features are columns
    precipitation = precipitation.T
    Tmean = Tmean.T
    
    # Calculate Potential Evapotranspiration (PET)
    PET = thornthwaite(Tmean)
    
    # Calculate the difference between precipitation and PET
    D = precipitation - PET
    
    # Calculate cumulative values every 8 days
    D_resampled = D.resample('8D', axis=1).sum()
    #D_resampled = D.T.resample('8D').sum().T (alternative resampling method, commented out)
    
    # Calculate SPEI
    def compute_spei(series, scale):
        """Calculate SPEI (Standardized Precipitation Evapotranspiration Index)"""
        # Cumulative sum
        cum_sum = series.cumsum()
        # Calculate mean and standard deviation
        mean = cum_sum.mean()
        std = cum_sum.std()
        # Standardization
        spei = (cum_sum - mean) / std
        return spei
    
    spei_values = D_resampled.apply(lambda x: compute_spei(x, scale=1), axis=1)
    spei_df = pd.DataFrame(data = spei_values.values,columns= [f'Week{week}_SPEI' for week in range(1, 47)])
    return spei_df

def thornthwaite(T, lat=45):
    """Thornthwaite method for Potential Evapotranspiration (PET) calculation"""
    I = (T / 5.0) ** 1.514
    a = (6.75e-7) * I**3 - (7.71e-5) * I**2 + (1.79e-2) * I + 0.49239
    PET = 16 * ((10 * T / I) ** a)
    return PET
    
def extreme_temperature(dates,Tmax,Tmin,T_upper,T_lower):
    # Calculate extreme temperature indicators (GDD, HDD, CDD)
    # GDD: Growing Degree Days; HDD: Heating Degree Days; CDD: Cooling Degree Days
    GDD = np.where(Tmax < T_lower, 0, (np.minimum(Tmax, T_upper) + np.maximum(Tmin, T_lower)) / 2 - T_lower)
    HDD = np.maximum(Tmax, T_upper) - T_upper
    CDD = np.minimum(Tmin, T_lower) - T_lower
    
    # Resample to 8-day periods and sum
    GDD = pd.DataFrame(GDD.T, index=dates).T.resample('8D', axis=1).sum()
    HDD = pd.DataFrame(HDD.T, index=dates).T.resample('8D', axis=1).sum()
    CDD = pd.DataFrame(CDD.T, index=dates).T.resample('8D', axis=1).sum()
    
    # Convert to DataFrames with standard column names
    CDD_df = pd.DataFrame(data = CDD.values,columns= [f'Week{week}_CDD' for week in range(1, 47)])
    HDD_df = pd.DataFrame(data = HDD.values,columns= [f'Week{week}_HDD' for week in range(1, 47)])
    GDD_df = pd.DataFrame(data = GDD.values,columns= [f'Week{week}_GDD' for week in range(1, 47)])
    return CDD_df,HDD_df,GDD_df  


def aggre_8days(dynamic_features,dates,data_new):
    # Aggregate dynamic feature data by 8-day periods
    data_new1 = pd.DataFrame()
    for feature in dynamic_features:
        # Select columns corresponding to the current dynamic feature
        columns = [col for col in data_new.columns if feature in col];
        data = data_new[columns];
        
        # Reindex to fill date gaps if the number of columns is less than the number of dates
        if data.shape[1]<len(dates):
            date_columns = pd.to_datetime(data.columns.str.replace(feature, ''), format='%Y_%m_%d')
            data = data.T
            data.index = date_columns
            full_index = pd.date_range(start=dates[0], end=dates[-1])
            data = data.reindex(full_index).sort_index()
        else:
            data = pd.DataFrame(data.T.values, index=dates)
        
        # Aggregate by 8 days: sum for precipitation, average for other features
        if feature =='_Pre':
            data = data.T.resample('8D', axis=1).sum()
            data.columns = [f'Week{week}_{feature[1:]}' for week in range(1, 47)]
        else:
            data = data.T.resample('8D', axis=1).mean()
            data.columns = [f'Week{week}_{feature[1:]}' for week in range(1, 47)]
        
        # Merge aggregated data to the result
        data_new1 = pd.concat([data_new1, data], axis=1)
    return data_new1


def extract_selected_variables(inputpath_base):
    # Read selected feature variables
    inpath_dates = os.path.join(inputpath_base, '01_data','05_buildmodel', '04_selectFeatures','selectFeatures.txt')
    # Construct file path
    gs_infornamtion = pd.read_csv(inpath_dates, sep='\t', header=None)
    gs_infornamtion.columns = ['slected_dynamic_features', 'slected_static', 'regionID']
    
    # Parse string-formatted lists to actual lists
    gs_infornamtion['slected_dynamic_features'] = gs_infornamtion['slected_dynamic_features'].apply(ast.literal_eval)
    gs_infornamtion['slected_static'] = gs_infornamtion['slected_static'].apply(ast.literal_eval)
    return gs_infornamtion

# Direct reading may reduce significant data loading time (original comment)
def find_weeks(inputpath_base,forecastDataList):
    # Find corresponding week numbers based on forecast dates
    
    # Read week-date mapping file
    file_path = os.path.join(inputpath_base, '02_S2S', '01_dataori', 'ECMWF','CommonYear_Week.txt')
    with open(file_path, 'r') as file:
        week_dates = [line.strip() for line in file.readlines()]
    result = []
    
    # Iterate through each date in forecastDataList
    for date in forecastDataList:
        # Iterate through week_dates to find the week of the date
        for i in range(len(week_dates) - 1):
            # Check if the date is within the current date range (inclusive of lower bound, exclusive of upper bound)
            if week_dates[i] <= date < week_dates[i + 1]:
                result.append((date, i + 1))  # Week 1 corresponds to index 0, so week number is i + 1
                break
        # Handle dates beyond the last date range (i.e., week 46 range)
        else:
            if date >= week_dates[-1]:
                result.append((date, len(week_dates)))  # Last week: week 46
    result = {date: week for date, week in result}
    return result

In [28]:
import os
import sys
import pandas as pd
import numpy as np
import xarray as xr
import rioxarray
import geopandas as gpd
from datetime import datetime, timedelta, date
root_directory = os.getcwd()[0:3]
sys.path.append(root_directory+'SCI\\SCI9_1\\01_code')
sys.path.append(r'C:\ProgramData\anaconda3\Lib\site-packages') 
sys.path.append(r'C:\Users\DELL\.conda\envs\myenv\Lib\site-packages') 

import warnings
warnings.filterwarnings("ignore")
import ast
from fastdtw import fastdtw
 
# Get current working directory
current_directory = os.getcwd()
print("Current working directory:", current_directory)
 
# Get the name of the current folder
current_folder_name = os.path.basename(current_directory)
print("Current folder name:", current_folder_name)
 
# Get the name of the parent folder
parent_directory = os.path.dirname(current_directory)
parent_folder_name = os.path.basename(parent_directory)
print("Parent folder name:", parent_folder_name)

当前工作目录: F:\SCI\SCI9_1\01_code\02_Wheat\06_India
当前文件夹名字: 06_India
上一级文件夹名字: 02_Wheat


In [29]:
# Variables that need to be modified
crop = parent_folder_name; countryID = current_folder_name; variable = 'mx2t6';
region = 'I';
country = countryID.split('_')[1]
############## Regional Configuration #############################################
inpath_dates_other = root_directory + '\\SCI\\SCI9_1\\02_data\\'+crop+'\\'+countryID+'\\'+'01_data'+'\\'+'07_Information'
other_infornamtion = pd.read_csv(os.path.join(inpath_dates_other,'information.txt'), sep=' ', header=None)
startyear, endyear, shp_name = other_infornamtion.iloc[0,0], other_infornamtion.iloc[0,1], other_infornamtion.iloc[0,2]

In [30]:
file_path = os.path.join(inputpath_base, '02_S2S', '01_dataori',institution,'CommonYear_Week.txt')
inpath_dates = os.path.join(inputpath_base, '01_data', '05_buildmodel', '02_extractdates', 'gs_three_periods.txt')
# Read lines from the file and strip whitespace from both ends
with open(file_path, 'r') as file:
    lines = [line.strip() for line in file.readlines()]

# Read start point and harvest point from another file
file_path = os.path.join(inputpath_base, '02_S2S', '01_dataori',institution,'CommonYear_Week.txt')
inpath_dates = os.path.join(inputpath_base, '01_data', '05_buildmodel', '02_extractdates', 'gs_three_periods.txt')
# Read lines from the file and strip whitespace from both ends
with open(file_path, 'r') as file:
    lines = [line.strip() for line in file.readlines()]

# Read start point and harvest point from another file
gs_infornamtion = pd.read_csv(inpath_dates, sep='\t', header=None)
gs_infornamtion.columns = ['start_point', 'peak', 'harvest_point','VI_select2','regions']
harvest_point = gs_infornamtion[gs_infornamtion['regions']==region]['harvest_point'].values[0]
start_point = gs_infornamtion[gs_infornamtion['regions']==region]['start_point'].values[0]
VI_select2 = gs_infornamtion[gs_infornamtion['regions']==region]['VI_select2'].values[0]

# Convert the string-type start point and harvest point to integers
harvest_point = int(harvest_point) # 8-day aggregation is backward-looking; for example, '01-01' refers to the aggregated indicator from 01-01 to 01-08
start_point = int(start_point)

# Get the corresponding dates from the 'lines' list based on the indices of the start point and harvest point
if harvest_point ==46:
    harvest_date_doy = '-'+'12-31'
else:
    harvest_date_doy = '-' + lines[harvest_point]
start_date_doy = '-' + lines[start_point]

In [15]:
harvest_point

46

In [31]:
regions = ['I']
Forecastyears = {'I': endyear}
# Define temperature thresholds according to crop type
if crop == '02_Wheat':
    T_upper = 34
    T_lower = 0
elif crop == '01_Maize':  # Fixed spelling error
    T_upper = 30
    T_lower = 8
elif crop == '03_Rice':
    T_upper = 35
    T_lower = 8    
else:
    T_upper = 30
    T_lower = 10
    


VIs =  ['_KNDVI' ,'_EVI','_NDVI']  # Vegetation Indices (KNDVI: Kernel NDVI, EVI: Enhanced Vegetation Index, NDVI: Normalized Difference Vegetation Index)
Cilmate = ['_Pre' ,'_Tmin' ,'_Solar','_Tmean','_Tmax']  # Climate variables (Pre: Precipitation, Tmin: Minimum Temperature, Solar: Solar Radiation, Tmean: Mean Temperature, Tmax: Maximum Temperature)
Climate_Exogenous  = ['_CDD' ,'_HDD' ,'_GDD','_VPD','_wind_speed','_SPEI'] # Exogenous climate variables (CDD: Cooling Degree Days, HDD: Heating Degree Days, GDD: Growing Degree Days, VPD: Vapor Pressure Deficit, SPEI: Standardized Precipitation Evapotranspiration Index)
soil_feature = [ 'SAND','AWC', 'SILT','ORG_CARBON',  'TOTAL_N', 'PH_WATER',  'CEC_SOIL', 'CLAY']  # Soil characteristic variables (SAND: Sand content, AWC: Available Water Capacity, SILT: Silt content, ORG_CARBON: Organic Carbon, TOTAL_N: Total Nitrogen, PH_WATER: Soil pH (in water), CEC_SOIL: Cation Exchange Capacity, CLAY: Clay content)
loc_feature = ['elevation', 'lat', 'lon']  # Location feature variables (elevation: Elevation, lat: Latitude, lon: Longitude)
Year_feature = ['year']; union_feature = ['idJoin'];  # Year feature and union key variable
dynamic_features = [ '_KNDVI' ,'_EVI','_NDVI','_Pre' ,'_Tmin' ,'_Solar','_Tmean','_VPD', '_wind_speed' ,'_Tmax']  # Dynamic feature variables (combination of VIs and core climate variables)
inputpath_base = root_directory + '\\SCI\\SCI9_1\\02_data\\'+crop+'\\'+countryID+'\\'  # Base input path

forecast_days = 46; country = countryID.split('_')[1]  # Forecast duration (46 weeks equivalent) and extract country code from countryID
yield_type = 'actual_yield'; origen ='ECMWF'; institution = 'ECMWF';  # Yield type (actual yield), data source (ECMWF: European Centre for Medium-Range Weather Forecasts)
ECMWF_path = os.path.join(inputpath_base,'02_S2S')  # S2S (Subseasonal to Seasonal) data path
S2S_data_path = os.path.join(ECMWF_path,'02_Reforecast', origen)  # Reforecast data path under S2S

In [4]:
shp_name

'Indiagee_up_load_winter.shp'

In [32]:
shp_all = os.path.join(inputpath_base,'01_data','02_shp',shp_name)
gdf_all = gpd.read_file(shp_all)


'''
#shp_name = 'Chinagee_up_load.shp'
#shp_all = os.path.join(inputpath_base,'01_data','02_shp',shp_name)
# Output sample variable TIF file for subsequent pointid reading preparation
gdf_all = gpd.read_file(shp_all)
if 'region' not in gdf_all.columns:
    gdf_all['region'] ='I'
else:
    pass
'''

for region in regions:
    if 'region' not in gdf_all.columns:
        gdf_all['region'] ='I'
        filtered_data_ori_upload = gdf_all
    else:
        filtered_data_ori_upload = gdf_all# [gdf_all['region']==region] # [ToDo] Verify if there are sub-regions
    Forecastyear =  Forecastyears[region]; years = str(Forecastyear)+'_'+str(Forecastyear);
    forecastDataList = os.listdir(os.path.join(inputpath_base,'02_S2S','02_Reforecast','ECMWF',region))
    filename = institution+"_"+variable+"_"+country+"_cf_forecasttimefcst_"+years+".grib"
    file = os.path.join(inputpath_base, '02_S2S','02_Reforecast','ECMWF',region,forecastDataList[0],filename)
    ds = xr.open_dataset(file, engine='cfgrib')
    d2m_data = ds.isel(step=0)['mx2t6']
    d2m_data.rio.write_crs("epsg:4326", inplace=True)  # Assume the data uses WGS84 coordinate system
    os.makedirs(os.path.join(inputpath_base,'02_S2S','08_Tif',region), exist_ok=True)
    output_path = os.path.join(inputpath_base,'02_S2S','08_Tif',region,region+'_'+variable+'_data.tif')
    d2m_data.rio.to_raster(output_path)
    # filtered_data_ori_upload = gdf_all[gdf_all['region']==region]
    filtered_data_ori_upload.to_file(os.path.join(inputpath_base, '01_data','02_shp',country+'_'+region+'.shp'), driver='ESRI Shapefile')

In [20]:
# [TODO] 确认是否已经准备好所有区的dataJoin#########确认好后继续往下处理
d2m_data

<xarray.DataArray 'mx2t6' (latitude: 18, longitude: 23)>
[414 values with dtype=float32]
Coordinates:
    number             int32 ...
    time               datetime64[ns] ...
    step               timedelta64[ns] 06:00:00
    heightAboveGround  float64 ...
  * latitude           (latitude) float64 36.4 34.9 33.4 31.9 ... 13.9 12.4 10.9
  * longitude          (longitude) float64 66.1 67.6 69.1 ... 96.1 97.6 99.1
    valid_time         datetime64[ns] ...
    spatial_ref        int32 0
Attributes: (12/30)
    GRIB_paramId:                             121
    GRIB_dataType:                            cf
    GRIB_numberOfPoints:                      414
    GRIB_typeOfLevel:                         heightAboveGround
    GRIB_stepUnits:                           1
    GRIB_stepType:                            max
    ...                                       ...
    GRIB_shortName:                           mx2t6
    GRIB_totalNumber:                         11
    GRIB_units:                               K
    long_name:                                Maximum temperature at 2 metres...
    units:                                    K
    standard_name:                            air_temperature

In [33]:
# TODO########################## Prepare dataJoin_I before running this code ##############################################
# Output to 03_outputData

# Forecast data processing
for region in regions:
    Forecastyear = Forecastyears[region]; years = str(Forecastyear)+'_'+str(Forecastyear);
    _, _, _, harvest_date_doy, _ = extract_dates(inputpath_base,institution,region)
    dataJoin1 =  pd.read_csv(os.path.join(inputpath_base, '02_S2S', '01_dataori',institution, f'dataJoin_{region}.csv'))
    forecastDataList = os.listdir(os.path.join(inputpath_base,'02_S2S','02_Reforecast','ECMWF',region))
    # Method 2: Suitable for large countries (20241104) - One grid point per administrative region
    # A total of 11 ensemble models, output by individual model
    for ii in forecastDataList:# Note: Modify here to batch processing algorithm #[0:1]
        ########################### Read harvest date ###################################
        pathto = os.path.join(ECMWF_path,'02_Reforecast', origen, region,ii)
        outpath = os.path.join(ECMWF_path,'03_outputData', origen, region,ii)
        create_directory(outpath)
        ds_pf_mx2t6, ds_pf_mn2t6, ds_pf_2t, ds_pf_tp, ds_df_wind_speed, ds_pf_vpd, ds_pf_ssr = read_pf_data(pathto, origen, country, years)
        ds_cf_mx2t6, ds_cf_mn2t6, ds_cf_2t, ds_cf_tp, ds_cf_wind_speed, ds_cf_vpd, ds_cf_ssr = read_cf_data(pathto, origen, country, years)
        
        ds_cf_mn2t6, ds_cf_mx2t6 = process_temperature_data(ds_cf_mn2t6, ds_cf_mx2t6)
        ds_pf_mn2t6, ds_pf_mx2t6 = process_temperature_data(ds_pf_mn2t6, ds_pf_mx2t6)
        ds_pf_mx2t6 = ds_pf_mx2t6[0]; ds_pf_mn2t6 = ds_pf_mn2t6[0];
        ds_cf_mx2t6 = ds_cf_mx2t6[0]; ds_cf_mn2t6 = ds_cf_mn2t6[0];
        ds_pf_mn2t6 = np.transpose(ds_pf_mn2t6, (1, 0, 2, 3))
        ds_pf_mx2t6 = np.transpose(ds_pf_mx2t6, (1, 0, 2, 3))
        ds_pf_2t, ds_pf_tp, ds_pf_ssr, ds_df_wind_speed, ds_pf_vpd = process_df_arrays(ds_pf_2t, ds_pf_tp, ds_pf_ssr, ds_df_wind_speed, ds_pf_vpd)
        ds_cf_2t, ds_cf_tp, ds_cf_ssr, ds_cf_wind_speed, ds_cf_vpd = process_cf_arrays(ds_cf_2t, ds_cf_tp, ds_cf_ssr, ds_cf_wind_speed, ds_cf_vpd)
        
        # Output of 10 PF (Probabilistic Forecast) models
        number = 10
        for ID in range(0, number):
            year = Forecastyear
            date_harvest = str(year) + harvest_date_doy
            date_harvest = pd.Timestamp(date_harvest).strftime('%Y%m%d')
            time_value = str(year) + '-' + ii  # Date of the first forecast day
            time_value1 = pd.Timestamp(time_value).strftime('%Y%m%d') # Date of the first forecast day
            days_between = calculate_days_between(time_value1, date_harvest)# Only extract data used for modeling
            steps_total = min(days_between + 1, forecast_days)
            features = ['Tmax', 'Tmin', 'Tmean', 'Pre', 'wind_speed', 'VPD', 'Solar']
            data_every_number = dataJoin1[['idJoin','pointid']]
            for feature, selected in zip(features, [ds_pf_mn2t6, ds_pf_mx2t6, ds_pf_2t, ds_pf_tp, ds_df_wind_speed, ds_pf_vpd, ds_pf_ssr]):
                for step in range(0, steps_total):  # Only extract data used for modeling; one extra day is extracted but has no impact (will not be used)
                    time = (pd.Timestamp(time_value1) + pd.Timedelta(days=step)).strftime('%Y%m%d') # Current forecast date (1st day, 2nd day, and so on)
                    data_sel = selected[ID, step, :].flatten()
                    data_every_number[time + '_' + feature] = data_every_number['pointid'].apply(lambda x: data_sel[x-1])
                    # Link data according to the position indicated by pointId in dataJoin, generate a new set of feature columns, and add them to the original pointId to form data_every_number
            data_every_number.to_csv(os.path.join(outpath, 'number' + str(ID) + '.csv'), index=False)
            
        # Output of the CF (Control Forecast) model (11th model)
        number = 10
        data_every_number = dataJoin1[['idJoin','pointid']]
        for feature, selected in zip(features, [ds_cf_mn2t6, ds_cf_mx2t6, ds_cf_2t, ds_cf_tp, ds_cf_wind_speed, ds_cf_vpd, ds_cf_ssr]):
            for step in range(0, steps_total):  # Only extract data used for modeling; one extra day is extracted but has no impact (will not be used)
                time = (pd.Timestamp(time_value1) + pd.Timedelta(days=step)).strftime('%Y%m%d') # Current forecast date (1st day, 2nd day, and so on)
                data_sel = selected[step, :].flatten()
                data_every_number[time + '_' + feature] = data_every_number['pointid'].apply(lambda x: data_sel[x-1])
        data_every_number.to_csv(os.path.join(outpath, 'number' + str(number) + '.csv'), index=False)

In [34]:
# TODO Verify if the format of pre_name is correct
# For the forecast year, output the ensemble of forecast data after the forecast date and the weekly-scale ensemble of historical 30 years, aggregated into 0-46 weeks
# Read the original forecast year and the advance forecast date

# Using direct reading may significantly reduce data loading time

#[20250317] The definition of simple_week is missing

numbers=11
for region in regions:
    pre_name ='WinterWheat'+'_'+countryID[3:]+'I_' # Need to verify if it conforms to this identifier format
    '''
    if region == 'I':
        pre_name = 'EarlyRice_daily'+country+region+'_'
    elif region == 'II':
        pre_name = 'LateRice_daily'+country+region+'_'
    else:
        pre_name = 'SingleRice_daily'+country+region+'_'
    '''
    Forecastyear = Forecastyears[region]
    years = range(startyear,Forecastyear) # Can be used to find similar KNDVI in historical years
    data_ori = pd.read_csv(os.path.join(inputpath_base, '01_data', '04_GEEdownloadData','01_DailyData',pre_name+str(Forecastyear))+'.csv')
    data_ori.set_index('idJoin', inplace=True)
    forecastDataList = os.listdir(os.path.join(inputpath_base,'02_S2S','02_Reforecast','ECMWF',region))
    
    # Read selected variables for subsequent variable filtering
    Forecastyear = Forecastyears[region]
    SelFeature_infornamtion = extract_selected_variables(inputpath_base)
    TimeFeatures_sel, Static_sel, regionID = SelFeature_infornamtion[SelFeature_infornamtion['regionID'] == region].iloc[0]
    
    # Actual number of weeks used for modeling
    inpath_dates = os.path.join(inputpath_base, '01_data','05_buildmodel', '02_extractdates','gs_three_periods.txt')
    gs_infornamtion = pd.read_csv(inpath_dates, delim_whitespace=True, header=None)
    gs_infornamtion.columns = ['start_point', 'peak', 'harvest_point', 'VI_select2','regionID']
    start_point, peak, harvest_point, VI_select2, region = gs_infornamtion[gs_infornamtion['regionID'] == region].iloc[0]
    
    # Data reading and index filtering
    data_ori_all = pd.read_csv(os.path.join(inputpath_base, '01_data','05_buildmodel','01_weekdata',region+'_allweekYielddata_VIs.csv'))
    Static_sel= [col for col in Static_sel if 'year.1' not in col] 
    TimeFeatures_sel_all= [col for col in data_ori_all.columns if any(feature in col for feature in TimeFeatures_sel)]
    TimeFeatures_sel_all= [col for col in TimeFeatures_sel_all if 'Previous_Yield' not in col] # Note: Yield from the previous year may be filtered due to 'pre' (precipitation) related columns; carefully verify
    filtered_columns_all = TimeFeatures_sel_all+Static_sel
    data_ori_all = data_ori_all[filtered_columns_all+['idJoin','Yield']] # Filter selected variables for subsequent analysis


        # Filter VIs for subsequent identification
    filtered_columns_VI = [col for col in data_ori_all.columns if VI_select2 in col]
    data_S2S_VI = data_ori_all[filtered_columns_VI + ['year','idJoin']]
    data_S2S_VI_mean = data_S2S_VI[filtered_columns_VI + ['year']].groupby('year').mean()
    
    S2SWeekList = ['leadweek_'+str(week) for week in range(1,6+1)] # 1 year in advance (i.e., 46 weeks)
    data_ori_current = data_ori_all[data_ori_all['year']==Forecastyear]



    dataweeks = find_weeks(inputpath_base,forecastDataList)
    
    unique_values = {}
    for key, value in dataweeks.items():
        if value not in unique_values.values():
            unique_values[key] = value
    
    simple_week = {key: f"leadweek_{len(dataweeks) - idx}" for idx, key in enumerate(unique_values.keys())}    
    for ii, leadweek in simple_week.items():# Note: Modify here to batch processing algorithm #[0:1]
        # Loop through model numbers
        for number in range(0,numbers):
            data_S2S_new = data_ori.copy()
            inputpath = os.path.join(ECMWF_path,'03_outputData', institution, region,ii)
            data_S2S = pd.read_csv(os.path.join(inputpath,'number'+ str(number) + '.csv'))
            data_S2S.set_index('idJoin', inplace=True)
            common_columns = data_S2S.columns.intersection(data_S2S_new.columns)
            data_S2S_new[common_columns] = data_S2S[common_columns] 
           # data_S2S_new.to_csv(os.path.join(outpath, str(year) + '.csv'))
            data_S2S_new = process_climate_data(data_S2S_new.reset_index(), Forecastyear, T_upper, T_lower, dynamic_features, soil_feature, loc_feature, Year_feature, union_feature)
            #data_S2S_new.to_csv(os.path.join(outpath, 'number'+str(number) + '.csv'),index=False)

            data_S2S_new_update = data_ori_current.copy()
            #data_his_new = data_his_new.merge(data_ori_current[filtered_columns_VI+Static_sel+['idJoin','Yield']],on='idJoin',how='inner')# Update VIs, static variables and Yield to ensure consistent data types
            # Perform variable category filtering to ensure only selected variables are included
            data_S2S_new['year'] = Forecastyear 
            # data_his_new = data_his_new[filtered_columns_all+['idJoin']]      

            ############################################## Find the most similar vegetation index for filling ############################################################################
            week_forecast = harvest_point+1-int(leadweek[9:])
            # From the previous year to the next year to ensure a full year of actual data (cannot only use the current year as actual data may be unavailable); from the current year to the current date
            forecast_weeklist1 = range(week_forecast, harvest_point + 1)
            forecast_weeklist = [f'Week{week}{VI_select2}' for week in forecast_weeklist1]# No forecast data for week_forecast (from the current forecast week to the harvest week)
            V1= [f'Week{week}{VI_select2}' for week in range(1, week_forecast)];
            V2= [f'Week{week}{VI_select2}' for week in range(week_forecast, 46+1)];
    
            current_S2S_VI_before =pd.concat([data_S2S_VI_mean.loc[Forecastyear][V1], data_S2S_VI_mean.loc[Forecastyear-1][V2]])
            dtw_distances = {}
            for year1 in range(startyear+1,Forecastyear):# Will not include Forecastyear
                other_S2S_VI_before = pd.concat([data_S2S_VI_mean.loc[year1][V1], data_S2S_VI_mean.loc[year1-1][V2]])
                distance, path = fastdtw(current_S2S_VI_before, other_S2S_VI_before)# Forecast from current week to harvest week
                dtw_distances[year1] = distance
            most_similar_by_dtw = min(dtw_distances, key=dtw_distances.get) # Find the most similar year (e.g., 2016 in the example)
            model_VI_replace = forecast_weeklist
                
        
                
            data_S2S_VI_forecast2 = data_S2S_VI[data_S2S_VI['year'] == most_similar_by_dtw][model_VI_replace+['idJoin']]# Only retain data needed for modeling
            data_S2S_new_update = data_S2S_new_update.drop(model_VI_replace,axis=1) # Delete columns corresponding to the forecast period; retain non-forecast columns
            data_S2S_new_update = data_S2S_new_update.merge(data_S2S_VI_forecast2,on='idJoin',how='inner')

            ############################################## Replace the forecast-period weeks in the original data with S2S forecast data ###################################################################
            # Loop to find variables that need to be replaced (replace original data with historical/S2S data)
            update_climate = []
            for feature in [feature for feature in TimeFeatures_sel if feature != VI_select2[1:]]: # Selected meteorological data excluding vegetation indices
                update_climate += [f'Week{week}_{feature}' for week in forecast_weeklist1]
            data_S2S_new_update.set_index('idJoin', inplace=True)
            data_S2S_new.set_index('idJoin', inplace=True)
            data_S2S_new_update[update_climate] = data_S2S_new[update_climate] # Replace original data with S2S forecast data

            ############################################## Filter variables for the growing season ############################################################################
            weeks = []
            # Determine if the growing season spans across years
            if start_point < harvest_point:  # Does not span years
                for feature in TimeFeatures_sel:
                    # Generate combinations of weeks and features using list comprehension
                    weeks += [f'Week{week}_{feature}' for week in range(start_point, harvest_point + 1)]
            else:  # Spans across years
                for feature in TimeFeatures_sel:
                    # Merge two date ranges and generate combinations of weeks and features
                    weeks += [f'Week{week}_{feature}' for week in list(range(start_point, 47)) + list(range(1, harvest_point + 1))]
            gs_features = weeks + Static_sel+['Yield']+['idJoin']
            data_S2S_new_update.reset_index('idJoin', inplace=True)            
            data_S2S_new_update = data_S2S_new_update[gs_features]
            
            ############################################## Output ############################################################################
            S2S_outputpath = os.path.join(inputpath_base,'02_S2S','05_WeekData','01_S2S','VI_Like',region,leadweek)
            os.makedirs(S2S_outputpath,exist_ok=True)
            data_S2S_new_update.to_csv(os.path.join(S2S_outputpath,'number'+ str(number) + '.csv'),index=False)

In [35]:
######################## Calculate similarity based on columns: ['number','Leadweek','hist_year','climate','distance'] #######################
############################ Supplement subsequent missing forecast data (excluding KNDVI) based on similar years #######################################
######################### Similar years are determined using all meteorological factors ##############################################
gs_infornamtion = pd.read_csv(inpath_dates, delim_whitespace=True, header=None)
gs_infornamtion.columns = ['start_point', 'peak', 'harvest_point', 'VI_select2','regionID']
start_point, peak, harvest_point, VI_select2, region = gs_infornamtion[gs_infornamtion['regionID'] == region].iloc[0]
if VI_select2[1:] in TimeFeatures_sel:
    TimeFeatures_sel.remove(VI_select2[1:])
hist_start_year = Forecastyear-30; hist_end_year = Forecastyear-1;
data = []
for year_hist in range(hist_start_year, hist_end_year+1):
    hist_outputpath1 = os.path.join(inputpath_base,'02_S2S','05_WeekData','02_hist',region)
    data_his_new_update = pd.read_csv(os.path.join(hist_outputpath1,'hist_'+str(year_hist)+'.csv'))
    dataweeks = find_weeks(inputpath_base,forecastDataList)
    
    unique_values = {}
    for key, value in dataweeks.items():
        if value not in unique_values.values():
            unique_values[key] = value
    
    simple_week = {key: f"leadweek_{len(dataweeks) - idx}" for idx, key in enumerate(unique_values.keys())}  
    
    for _, S2S_leadweek in simple_week.items():# Note: Modify here to batch processing algorithm #[0:1]
        if int(S2S_leadweek[9:])-6 <= 0:
            pass # No need to supplement with historical similar years
        else:
            # Loop through model numbers
            for number in range(0, numbers):
                if start_point < harvest_point: # Growing season in the same year
                    WeekList_len = len(list(range(1, harvest_point - start_point + 1)))# Note: The configured hisWeekList does not seem to include the start_point week
                    forecast_point = start_point + WeekList_len - int(S2S_leadweek[9:]) + 1
                    end_point = forecast_point + 6  # Excludes the last week
                    weeklist_keep = list(range(start_point, end_point))
                    weeklist_hiscom = list(range(end_point, harvest_point + 1))
                else:
                    WeekList_len = len(range(1, harvest_point - start_point + 1 + 46)) + len(range(1, harvest_point - start_point + 1))
                    forecast_point = start_point + WeekList_len - int(S2S_leadweek[9:]) + 1
                    end_point = forecast_point + 6 - 46 if forecast_point + 6 > 46 else forecast_point + 6
                    weeklist_keep = list(range(start_point, 47)) + list(range(1, end_point)) if start_point > end_point else list(range(start_point, end_point))
                    weeklist_hiscom = list(range(end_point, 47)) + list(range(1, harvest_point + 1)) if end_point > harvest_point else list(range(end_point, harvest_point + 1))
                    
    
                S2S_outputpath = os.path.join(inputpath_base,'02_S2S','05_WeekData','01_S2S','VI_Like',region,leadweek)
                data_S2S_new_update = pd.read_csv(os.path.join(S2S_outputpath,'number'+ str(number) + '.csv'))
                for climate1 in TimeFeatures_sel:
                    weeklist_keep_feature = [f'Week{week}_{climate1}' for week in weeklist_keep]
                    weeklist_hiscom_feature = [f'Week{week}_{climate1}' for week in weeklist_hiscom]
                    weeklist_keep_feature_mean_hist = data_his_new_update[weeklist_keep_feature].mean()
                    weeklist_keep_feature_mean_S2S = data_S2S_new_update[weeklist_keep_feature].mean()
                    distance, path = fastdtw(weeklist_keep_feature_mean_S2S, weeklist_keep_feature_mean_hist)# Forecast from current week to harvest week
                    data.append([number, S2S_leadweek, year_hist, climate1, distance]) 
                    
dtw_distances = pd.DataFrame(data, columns=['number','Leadweek','hist_year','climate','distance'])            
dtw_distances.to_csv(os.path.join(inputpath_base,'02_S2S','similaryears_finds.csv'),index=False)

In [36]:
# 按照所以选择的指标计算一个最相似的年份，补充预报后面的数据#############################
hist_start_year = Forecastyear-30;hist_end_year = Forecastyear-1;
min_distance_idx = dtw_distances.groupby(['number', 'Leadweek'])['distance'].idxmin()
# 通过索引获取对应的 hist_year 和其他列（如果需要的话）
min_distance_years = dtw_distances.loc[min_distance_idx, ['number', 'Leadweek', 'hist_year', 'distance']]


dtw_distances = pd.read_csv(os.path.join(inputpath_base,'02_S2S','similaryears_finds.csv'))
dataweeks = find_weeks(inputpath_base,forecastDataList)

unique_values = {}
for key, value in dataweeks.items():
    if value not in unique_values.values():
        unique_values[key] = value

simple_week = {key: f"leadweek_{len(dataweeks) - idx}" for idx, key in enumerate(unique_values.keys())}   
for _, S2S_leadweek in simple_week.items():# 注意修改这里改成批量算法 #[0:1]
    S2S_outputpath1 = os.path.join(inputpath_base,'02_S2S','06_buildmodel','01_S2S','VI_Like',region,S2S_leadweek)
    os.makedirs(S2S_outputpath1,exist_ok=True)
    if int(S2S_leadweek[9:])-6<=0:
        for number in range(0,numbers):
            S2S_outputpath = os.path.join(inputpath_base,'02_S2S','05_WeekData','01_S2S','VI_Like',region,S2S_leadweek)
            data_S2S_new_update= pd.read_csv(os.path.join(S2S_outputpath,'number'+ str(number) + '.csv'))
            S2S_outputpath1 = os.path.join(inputpath_base,'02_S2S','06_buildmodel','01_S2S','VI_Like',region,S2S_leadweek)
            data_S2S_new_update.to_csv(os.path.join(S2S_outputpath1,'number'+ str(number) + '.csv'))  
    else:
        # for number循环
        for number in range(0,numbers):
            if start_point < harvest_point: # 同年生长
                WeekList_len = len(list(range(1,harvest_point-start_point+1)))# 设定的hisWeekList好像不包括了start_point周
                forecast_point = start_point+WeekList_len-int(S2S_leadweek[9:])+1
                end_point = forecast_point+6  # 不包括最后一周
                weeklist_keep =  list(range(start_point,end_point))
                weeklist_hiscom =  list(range(end_point,harvest_point+1))
            else:
                WeekList_len =len(range(1,harvest_point-start_point+1+46))+len(range(1,harvest_point-start_point+1))
                forecast_point = start_point+WeekList_len-int(S2S_leadweek[9:])+1
                end_point = forecast_point + 6 - 46 if forecast_point + 6 > 46 else forecast_point + 6
                weeklist_keep =  list(range(start_point,47))+list(range(1,end_point))  if start_point > end_point else list(range(start_point,end_point))
                weeklist_hiscom = list(range(end_point,47))+list(range(1,harvest_point+1))  if end_point > harvest_point else list(range(end_point,harvest_point+1))
                
    
            S2S_outputpath = os.path.join(inputpath_base,'02_S2S','05_WeekData','01_S2S','VI_Like',region,S2S_leadweek)
            data_S2S_new_update= pd.read_csv(os.path.join(S2S_outputpath,'number'+ str(number) + '.csv'))
            similar_years = int(min_distance_years[(min_distance_years['number']==number)&(min_distance_years['Leadweek']==S2S_leadweek)]['hist_year'].values)
            hist_outputpath1 = os.path.join(inputpath_base,'02_S2S','05_WeekData','02_hist',region)
            data_his_new_update = pd.read_csv(os.path.join(hist_outputpath1,'hist_'+str(similar_years)+'.csv'))
            data_S2S_new_update_copy = data_S2S_new_update.copy()
            data_S2S_new_update_copy = data_S2S_new_update.set_index(['idJoin']);
            data_his_new_update = data_his_new_update.set_index(['idJoin']);
            weeklist_hiscom = ['Week'+str(week)+'_' for week in weeklist_hiscom]
            weeklist_hiscom_all= [col for col in data_S2S_new_update.columns if any(feature in col for feature in weeklist_hiscom)]
            weeklist_hiscom_all = [item for item in weeklist_hiscom_all if VI_select2 not in item]
            
            data_S2S_new_update_copy[weeklist_hiscom_all] = data_his_new_update[weeklist_hiscom_all];
           # S2S_outputpath = os.path.join(inputpath_base,'02_S2S','06_buildmodel','01_S2S','VI_Like',region,leadweek)
           # os.makedirs(S2S_outputpath,exist_ok=True)

            data_S2S_new_update_copy.to_csv(os.path.join(S2S_outputpath1,'number'+ str(number) + '.csv'))        

In [ ]:
################寻找最相似的前五年的#############################
min_distance_years_top_5 = dtw_distances[['number','Leadweek','hist_year','distance']].groupby(['number', 'Leadweek']).apply(
    lambda group: group.nsmallest(5, 'distance')
).reset_index(drop=True)
min_distance_years_top_5[(min_distance_years_top_5['number']==0)&(min_distance_years_top_5['Leadweek']=='leadweek_10')]

dtw_distances

In [219]:
os.path.join(hist_outputpath1,'hist_'+str(similar_years)+'.csv')

'F:\\\\SCI\\SCI9_1\\02_data\\02_Wheat\\09_European\\02_S2S\\05_WeekData\\02_hist\\I\\hist_1992.csv'

In [234]:
# lead_8  45, 46, 1, 2, 3, 4
# lead_9  44, 45, 46, 1, 2, 3


In [225]:
data_his_new_update[weeklist_hiscom_all]

,Week19_SPEI,Week20_SPEI,Week21_SPEI,Week22_SPEI,Week23_SPEI,Week24_SPEI,Week25_SPEI,Week26_SPEI,Week27_SPEI,Week28_SPEI,...,Week19_Pre,Week20_Pre,Week21_Pre,Week22_Pre,Week23_Pre,Week24_Pre,Week25_Pre,Week26_Pre,Week27_Pre,Week28_Pre
idJoin,,,,,,,,,,,,,,,,,,,,,
AT32,0.408049,0.317654,0.230743,0.134998,0.044302,-0.046029,-0.137463,-0.226015,-0.316863,-0.404500,...,25.071520,41.087120,71.494306,50.573064,35.502615,49.802822,28.829998,64.776065,31.749152,69.990794
AT11,0.351903,0.272603,0.197916,0.125513,0.044137,-0.030291,-0.111182,-0.189610,-0.270201,-0.353404,...,4.850283,17.734279,45.574093,46.512334,6.481053,107.623962,10.463139,36.278669,2.860229,13.403733
AT33,0.466780,0.374567,0.284014,0.178718,0.086796,-0.004516,-0.096610,-0.189015,-0.280840,-0.369457,...,23.096828,60.722902,71.302267,48.175932,41.957419,57.514868,38.200027,51.286408,37.840401,73.517192
AT34,0.429006,0.339865,0.252458,0.153270,0.063298,-0.025785,-0.116760,-0.207941,-0.298268,-0.385554,...,28.317897,75.382253,80.504027,55.899956,41.167696,56.112149,30.839067,42.011143,34.130929,70.382639
AT22,0.362058,0.279639,0.201353,0.124048,0.039905,-0.038841,-0.122426,-0.201726,-0.284262,-0.368056,...,17.432355,21.846918,52.653036,45.285905,13.799507,83.794351,17.805066,59.125597,20.082834,34.459557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SE12,0.369023,0.293792,0.216666,0.139246,0.061359,-0.020481,-0.102164,-0.181437,-0.267401,-0.345071,...,6.343060,32.012591,27.270656,14.774992,21.969098,8.994144,18.501852,15.697783,0.800087,22.877628
SE31,0.393021,0.316925,0.237848,0.158581,0.081863,-0.000945,-0.083704,-0.163548,-0.250621,-0.329829,...,8.032646,43.882330,26.354470,16.036839,53.360216,14.380335,25.319454,27.117401,0.571944,27.397082
SE23,0.341699,0.268817,0.193989,0.119873,0.045907,-0.032701,-0.111864,-0.188642,-0.271562,-0.347819,...,10.691078,37.226896,21.998423,28.406346,43.895362,19.456904,19.244330,21.156696,0.719964,33.797604


In [227]:
data_S2S_new_update.set_index(['idJoin'])[weeklist_hiscom_all]

,Week19_SPEI,Week20_SPEI,Week21_SPEI,Week22_SPEI,Week23_SPEI,Week24_SPEI,Week25_SPEI,Week26_SPEI,Week27_SPEI,Week28_SPEI,...,Week19_Pre,Week20_Pre,Week21_Pre,Week22_Pre,Week23_Pre,Week24_Pre,Week25_Pre,Week26_Pre,Week27_Pre,Week28_Pre
idJoin,,,,,,,,,,,,,,,,,,,,,
AT11,0.378546,0.299539,0.214343,0.136893,0.051822,-0.028318,-0.104630,-0.189960,-0.268945,-0.339080,...,43.052968,5.913522,4.578223,56.943066,6.227609,5.700843,25.590212,20.853666,32.635587,36.962891
AT12,0.372131,0.293921,0.210468,0.132705,0.048054,-0.031043,-0.106907,-0.191283,-0.269091,-0.351541,...,30.556206,13.305013,12.525048,46.673459,5.005720,4.043101,24.757369,21.770578,36.404958,8.632812
AT13,0.377216,0.299195,0.214661,0.137157,0.052376,-0.027251,-0.102619,-0.187918,-0.266642,-0.368250,...,27.988845,13.013152,10.023957,53.101030,6.708251,2.404604,31.635641,17.140076,30.350946,39.494141
AT21,0.344479,0.265866,0.187785,0.111173,0.028251,-0.049602,-0.126448,-0.208941,-0.283995,-0.353736,...,55.339939,7.312349,34.588423,47.550913,24.407311,26.520981,27.039461,23.984824,62.519919,42.664063
AT22,0.359433,0.280675,0.198703,0.121564,0.038662,-0.038861,-0.114958,-0.199031,-0.274762,-0.355612,...,62.430359,9.096199,11.554838,43.601207,19.967512,21.879918,27.018040,17.007322,53.959546,17.478515
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SE31,0.348896,0.272956,0.197666,0.120931,0.043587,-0.031823,-0.108883,-0.191004,-0.270605,-0.333621,...,31.375346,26.210009,24.781536,19.667035,12.529726,19.681955,19.963743,14.792492,14.670331,26.217773
SK01,0.385434,0.306823,0.222257,0.143939,0.058719,-0.021740,-0.098224,-0.182930,-0.263245,-0.342011,...,37.278366,11.012713,10.027587,50.345584,3.058030,2.959487,22.625322,22.536374,22.714922,27.515625
SK02,0.378208,0.300011,0.215970,0.137516,0.052806,-0.027222,-0.103220,-0.185984,-0.265881,-0.344494,...,43.579727,11.488854,12.445776,45.730191,2.859425,1.495537,21.852090,27.156459,26.103275,18.910156
